# How to handle API keys and other user-specific parameters

To request to the OpenAI (and Azure OpenAI Service), we need to specify API token for authentication. And some scenarios also additional parameters such as organization IDs and Azure resource names must be provided.

This guide illustrates how the PSOpenAI handle these user-specific parameters.

In [ ]:
# imports
Import-Module ..\PSOpenAI.psd1 -Force

## API key

### 1. Set an environment variable `OPENAI_API_KEY`

Set the API key to the environment variable named `OPENAI_API_KEY`. PSOpenAI will use the key implicitly.

This method is best suited when running on a trusted host or CI/CD pipeline.

In [ ]:
$env:OPENAI_API_KEY = '<Put your API key here>'
Request-ChatGPT -Message "Who are you?" -Verbose >$null

VERBOSE: API Key found in environment variable "OPENAI_API_KEY".
VERBOSE: API key to be used is sk-I0I*******************************************B5
VERBOSE: Organization-ID to be used is 
VERBOSE: Request to OpenAI API
VERBOSE: HTTP/1.1 POST with 79-byte payload
VERBOSE: received 330-byte response of content type application/json
VERBOSE: OpenAI API response: 
StatusCode    : 200
processing_ms : 1109
request_id    : 54be116c347d2b42e5db89133c074442


### 2. Set a global variable `OPENAI_API_KEY`

Set the API key to the `$global:OPENAI_API_KEY` variable. PSOpenAI will use the key implicitly.

In [ ]:
$global:OPENAI_API_KEY = '<Put your API key here>'
Request-ChatGPT -Message "Who are you?" -Verbose >$null

VERBOSE: API Key found in global variable "OPENAI_API_KEY".
VERBOSE: API key to be used is sk-I0I*******************************************B5
VERBOSE: Organization-ID to be used is 
VERBOSE: Request to OpenAI API
VERBOSE: HTTP/1.1 POST with 79-byte payload
VERBOSE: received 441-byte response of content type application/json
VERBOSE: OpenAI API response: 
StatusCode    : 200
processing_ms : 2064
request_id    : 88f4fd28dc78917b0a17626c1b5afa31


### 3. Specify to the named parameter.

Specify the API key explicitly in the `-ApiKey` parameter. It must be specified each time the function is called.

This is best used when the function is called only once or with few calls, such as when executing manually from the console.

In [ ]:
Request-ChatGPT -Message "Who are you?" -ApiKey '<Put your API key here>' -Verbose >$null

VERBOSE: API key to be used is sk-I0I*******************************************B5
VERBOSE: Organization-ID to be used is 
VERBOSE: Request to OpenAI API
VERBOSE: HTTP/1.1 POST with 79-byte payload
VERBOSE: received 469-byte response of content type application/json
VERBOSE: OpenAI API response: 
StatusCode    : 200
processing_ms : 2558
request_id    : 75711e5b9b0a79d5113a984dcc748dcb


### Tips: API key as SecureString

You can set the type of API keys as `[string]` or `[SecureString]`.

SecureString can be used to mask user input and store credentials rather secure.

⚠️ Note: Do not use SecureString on macOS and Linux. SecureString is not encrypted on these platforms.

In [ ]:
# Get API key from user input
$SecureApiKey = Read-Host -Prompt 'Enter API key' -AsSecureString
# API key is stored securely
echo "API key is $SecureApiKey"

# Save SecureString to global variable.
$SecuredText = $SecureApiKey | ConvertFrom-SecureString
$SecuredText | Out-File 'EncodedAPIKey.txt' -Force

# This text can only be decrypted with
#   the combination of the computer and user account used for encryption.
echo "Secured API key text: $SecuredText"


API key is System.Security.SecureString
Secured API key text: 01000000d08c9ddf0115d1118c7a00c04fc297eb010000002343901df2c22b4db1ccc0a35e0258020000000002000000000003660000c0000000100000001a8bc4a28a275949832108e65096620e0000000004800000a0000000100000009abbb7b7b3cc99b8f8f21cee5784629f08000000ca17bc7322414068140000000f2aaafa6e13b3f7c15b474d0d2b3634da53dc2b


In [ ]:
# Let's use the encrypted API key for authentication
# read encrypted key from text file.
$SecuredText = Get-Content -Path 'EncodedAPIKey.txt'
$SecureApiKey = $SecuredText | ConvertTo-SecureString
Request-ChatGPT -Message "Who are you?" -ApiKey $SecureApiKey -Verbose >$null

VERBOSE: API key to be used is sk-I0I*******************************************B5
VERBOSE: Organization-ID to be used is 
VERBOSE: Request to OpenAI API
VERBOSE: HTTP/1.1 POST with 79-byte payload
VERBOSE: received 432-byte response of content type application/json
VERBOSE: OpenAI API response: 
StatusCode    : 200
processing_ms : 2031
request_id    : c71fafeb4961fba748d0f5e4a80c036f


## Organization ID

For users who belong to multiple organizations, you can pass an organization id for an API request. Usage from these API requests will count against the specified organization's subscription quota.

The method of specifying the organization ID is the same as for API keys. It can be specified as an environment variable, a global variable, or a named parameter. The variable name is `OPENAI_ORGANIZATION` and the parameter name is `-Orgnization`.

Note that SecureString cannot be used for organization ID. This is because there is usually no security risk even if the organization ID is exposed to public.

In [ ]:
# 1. Environment variable
$env:OPENAI_ORGANIZATION = '<Put your organization ID here>'

# 2. Global variable
$global:OPENAI_ORGANIZATION = '<Put your organization ID here>'

# 3. Named parameter
Request-ChatGPT -Message 'Hello OpenAI' -Organization '<Put your organization ID here>' -Verbose >$null

VERBOSE: API Key found in global variable "OPENAI_API_KEY".
VERBOSE: API key to be used is sk-I0I*******************************************B5
VERBOSE: Organization-ID to be used is org-BX********************IE
VERBOSE: Request to OpenAI API
VERBOSE: HTTP/1.1 POST with 79-byte payload
VERBOSE: received 319-byte response of content type application/json
VERBOSE: OpenAI API response: 
StatusCode    : 200
processing_ms : 1247
request_id    : a066189ca6e29d3668185f097c9b35f8


## API Base (Azure OpenAI Service)

For Azure OpenAI Service, the API base is required to be specified. This is usually a URL of the form `https://<your-resource-name>.openai.azure.com/`.

The method of specifying the API base is the same as for API keys. It can be specified as an environment variable, a global variable, or a named parameter. The variable name is `OPENAI_API_BASE` and the parameter name is `-ApiBase`.

Note that SecureString cannot be used for API base. This is because there is usually no security risk even if that is exposed to public.

In [ ]:
# Set required params for the Azure OpenAI Service
$global:OPENAI_API_KEY = '<Put your API key for Azure here>'
$Deployment = 'gpt35turbo-48c5d8d9aaa'

# 1. Environment variable
$env:OPENAI_API_BASE = 'https://your-resource-name.openai.azure.com/'

# 2. Global variable
$global:OPENAI_API_BASE = 'https://your-resource-name.openai.azure.com/'

# 3. Named parameter
Request-AzureChatGPT -Message 'Hello Azure' -Deployment $Deployment -ApiBase 'https://your-resource-name.openai.azure.com/' -Verbose >$null

VERBOSE: API Key found in global variable "OPENAI_API_KEY".
VERBOSE: API key to be used is 81c***************************e8
VERBOSE: Request to Azure API
VERBOSE: HTTP/1.1 POST with 54-byte payload
VERBOSE: received 325-byte response of content type application/json
VERBOSE: Azure API response: 
StatusCode    : 200
processing_ms : 555.6054
request_id    : 8f4622be-6b55-4462-bb37-ed1a8d9100e6
